In [1]:
%matplotlib inline
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt

wine_path = '../data/ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

wineq = torch.from_numpy(wineq_numpy)
wine_infos = wineq[:, :-1]
wine_quality = wineq[:, -1]

wine_infos, wine_infos.shape, wine_quality, wine_quality.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]),
 tensor([6., 6., 6.,  ..., 6., 7., 6.]),
 torch.Size([4898]))

In [2]:
n_samples = wineq.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_wine_infos = wine_infos[:-n_val]
train_wine_quality = wine_quality[:-n_val]
val_wine_infos = wine_infos[-n_val:]
val_wine_quality = wine_quality[-n_val:]

train_wine_infos.shape, train_wine_quality.shape, val_wine_infos.shape, val_wine_quality.shape

(torch.Size([3919, 11]),
 torch.Size([3919]),
 torch.Size([979, 11]),
 torch.Size([979]))

In [3]:
class WineModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden_linear = nn.Linear(11, 20)
        self.output_linear  = nn.Linear(20, 1)
        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = torch.tanh(hidden_t)
        output_t = self.output_linear(activated_t)
        
        return output_t

In [4]:
def train(n_epochs, optimizer, model, loss_fn, train_wine_infos, train_wine_quality, val_wine_infos, val_wine_quality):
    for epoch in range(1, n_epochs + 1):
        train_wine_pred = model(train_wine_infos)
        train_loss = loss_fn(train_wine_pred, train_wine_quality)
        
        with torch.no_grad():
            val_wine_pred = model(val_wine_infos)
            val_loss = loss_fn(val_wine_pred, val_wine_quality)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch == 1 or epoch % 500 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, train_loss, val_loss))
            print('\tTrain {}'.format(train_wine_pred.shape))
            print('\tVal   {}'.format(val_wine_pred.shape))

In [5]:
wine_model = WineModel()
learning_rate = 1e-5
optimizer = optim.SGD(wine_model.parameters(), lr=learning_rate)

import time
tic = time.time()

train(
    n_epochs=5000,
    optimizer=optimizer,
    model=wine_model,
    loss_fn=nn.MSELoss(),
    train_wine_infos=train_wine_infos,
    train_wine_quality=train_wine_quality,
    val_wine_infos=val_wine_infos,
    val_wine_quality=val_wine_quality)

toc = time.time()
print('Spend time ' + str(toc - tic) + 's')

/home/ivan/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([3919])) that is different to the input size (torch.Size([3919, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/ivan/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([979])) that is different to the input size (torch.Size([979, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Training loss 40.24391555786133, Validation loss 39.99428939819336
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 500, Training loss 26.296884536743164, Validation loss 25.917959213256836
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 1000, Training loss 17.55152702331543, Validation loss 17.218263626098633
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 1500, Training loss 11.865754127502441, Validation loss 11.509507179260254
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 2000, Training loss 8.059080123901367, Validation loss 7.76265287399292
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 2500, Training loss 5.589808940887451, Validation loss 5.297574996948242
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 3000, Training loss 3.9648818969726562, Validation loss 3.679269790649414
	Train torch.Size([3919, 1])
	Val   torch.Size([979, 1])
Epoch 3500, Training loss 2.894589424133301, Va